In [ ]:
# === SELF-CONTAINED SECOND PANE: Heatmaps + CSV summary from existing patch RMSEs ===
import os
import glob
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from tqdm import tqdm

# === CONFIG ===
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
INPUT_SIZE = 128
PATCH_STRIDE = 256
BATCH_SIZE = 16

TEST_FOLDER = r"C:\Users\shpigel-lab\Desktop\tiles\test"
MODEL_PATH = r"C:\Users\shpigel-lab\Documents\DL project\models\autoencoder.pth"
OUTPUT_CSV = r"C:\Users\shpigel-lab\Documents\DL project\test_anomalies.csv"
HEATMAP_DIR = r"C:\Users\shpigel-lab\Documents\DL project\test_heatmaps"
os.makedirs(HEATMAP_DIR, exist_ok=True)

# === TRANSFORM ===
transform = transforms.Compose([
    transforms.Resize((INPUT_SIZE, INPUT_SIZE)),
    transforms.ToTensor()
])

# === MODEL ===
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, 2, 1), nn.ReLU(),
            nn.Conv2d(16, 32, 3, 2, 1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, 2, 1), nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 3, 2, 1, output_padding=1), nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, 2, 1, output_padding=1), nn.ReLU(),
            nn.ConvTranspose2d(16, 3, 3, 2, 1, output_padding=1), nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))

model = Autoencoder().to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

# === Re-run inference just to get patch scores again ===
def process_group(folder):
    result = {}
    files = glob.glob(os.path.join(folder, "*.png"))
    batch = []
    coords = []

    for f in tqdm(files, desc="🔄 Loading & scoring test patches"):
        fname = os.path.basename(f)
        slide = fname.split("_")[0]
        x, y = int(fname.split("_")[1]), int(fname.split("_")[2].split(".")[0])
        try:
            img = Image.open(f).convert("RGB")
        except:
            continue
        tensor = transform(img)
        batch.append(tensor)
        coords.append((slide, x, y))

        if len(batch) == BATCH_SIZE:
            batch_tensor = torch.stack(batch).to(DEVICE)
            with torch.no_grad():
                recon = model(batch_tensor)
                rmse = torch.sqrt(torch.mean((batch_tensor - recon) ** 2, dim=(1, 2, 3))).cpu().numpy()
            for (sid, x, y), score in zip(coords, rmse):
                result.setdefault(sid, []).append((x, y, score))
            batch, coords = [], []

    if batch:
        batch_tensor = torch.stack(batch).to(DEVICE)
        with torch.no_grad():
            recon = model(batch_tensor)
            rmse = torch.sqrt(torch.mean((batch_tensor - recon) ** 2, dim=(1, 2, 3))).cpu().numpy()
        for (sid, x, y), score in zip(coords, rmse):
            result.setdefault(sid, []).append((x, y, score))

    return result

results = process_group(TEST_FOLDER)
np.save(r"C:\Users\shpigel-lab\Documents\DL project\test_patch_scores.npy", results)



In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

# === CONFIG ===
PATCH_STRIDE = 256
HEATMAP_DIR = r"C:\Users\shpigel-lab\Documents\DL project\test_heatmaps"
OUTPUT_CSV = r"C:\Users\shpigel-lab\Documents\DL project\test_anomalies.csv"
os.makedirs(HEATMAP_DIR, exist_ok=True)

# === LOAD RESULTS ===
results = np.load(r"C:\Users\shpigel-lab\Documents\DL project\test_patch_scores.npy", allow_pickle=True).item()

# === GENERATE HEATMAPS + CSV ===
rows = []
for sid, patches in results.items():
    try:
        xs, ys, scores = zip(*patches)
    except ValueError:
        print(f"⚠️ Skipping scan {sid}: no valid patches.")
        continue

    mean_score = np.mean(scores)
    rows.append({"Scan": sid, "Anomaly": mean_score})

    # Build color heatmap
    grid_w = (max(xs) // PATCH_STRIDE) + 1
    grid_h = (max(ys) // PATCH_STRIDE) + 1
    grid = np.zeros((grid_h, grid_w, 3), dtype=np.uint8)
    norm_scores = [(s - min(scores)) / (max(scores) - min(scores) + 1e-8) for s in scores]

    for (x, y, s) in zip(xs, ys, norm_scores):
        i, j = y // PATCH_STRIDE, x // PATCH_STRIDE
        color = (np.array(plt.cm.jet(s)[:3]) * 255).astype(np.uint8)
        grid[i, j] = color

    # Save heatmap
    heatmap = cv2.resize(grid, (grid_w * 10, grid_h * 10), interpolation=cv2.INTER_NEAREST)
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(heatmap)
    ax.set_title(f"{sid} heatmap")
    ax.axis('off')
    sm = plt.cm.ScalarMappable(cmap='jet', norm=plt.Normalize(vmin=min(scores), vmax=max(scores)))
    sm.set_array([])
    fig.colorbar(sm, ax=ax, fraction=0.04, pad=0.05)
    out_path = os.path.join(HEATMAP_DIR, f"{sid}_heatmap.png")
    plt.savefig(out_path, bbox_inches='tight')
    plt.close()
    print(f"✅ Saved heatmap: {sid} -> {out_path}")

# Save CSV
df = pd.DataFrame(rows)
df.to_csv(OUTPUT_CSV, index=False)
print(f"📊 Done! CSV saved to: {OUTPUT_CSV}")


In [ ]:
import matplotlib.pyplot as plt
import cv2

import os
import glob
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from tqdm import tqdm

import torch.nn as nn

# === DEVICE & TRANSFORM ===
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
INPUT_SIZE = 128
BATCH_SIZE = 16

transform = transforms.Compose([
    transforms.Resize((INPUT_SIZE, INPUT_SIZE)),
    transforms.ToTensor()
])

# === Autoencoder Model (same as before) ===
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, 2, 1), nn.ReLU(),
            nn.Conv2d(16, 32, 3, 2, 1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, 2, 1), nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 3, 2, 1, output_padding=1), nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, 2, 1, output_padding=1), nn.ReLU(),
            nn.ConvTranspose2d(16, 3, 3, 2, 1, output_padding=1), nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))

# === Load model ===
MODEL_PATH = r"C:\Users\shpigel-lab\Documents\DL project\models\autoencoder.pth"
model = Autoencoder().to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

# === Patch Scoring Function ===
def compute_patch_scores(folder):
    results = []
    files = glob.glob(os.path.join(folder, "*.png"))
    batch = []
    coords = []

    for f in tqdm(files, desc=f"Scoring {os.path.basename(folder)}"):
        fname = os.path.basename(f)
        slide = fname.split("_")[0]
        x, y = int(fname.split("_")[1]), int(fname.split("_")[2].split(".")[0])
        try:
            img = Image.open(f).convert("RGB")
        except Exception as e:
            print(f"⚠️ Failed to open {f}: {e}")
            continue
        tensor = transform(img)
        batch.append(tensor)
        coords.append((slide, x, y))

        if len(batch) == BATCH_SIZE:
            batch_tensor = torch.stack(batch).to(DEVICE)
            with torch.no_grad():
                recon = model(batch_tensor)
                rmse = torch.sqrt(torch.mean((batch_tensor - recon) ** 2, dim=(1, 2, 3))).cpu().numpy()
            for (slide_id, x, y), s in zip(coords, rmse):
                results.append((slide_id, x, y, s))
            batch = []
            coords = []

    # remaining
    if batch:
        batch_tensor = torch.stack(batch).to(DEVICE)
        with torch.no_grad():
            recon = model(batch_tensor)
            rmse = torch.sqrt(torch.mean((batch_tensor - recon) ** 2, dim=(1, 2, 3))).cpu().numpy()
        for (slide_id, x, y), s in zip(coords, rmse):
            results.append((slide_id, x, y, s))

    return results

In [ ]:
BASELINE_PATH = r"C:\Users\shpigel-lab\Documents\DL project\baseline_test_rmse.txt"
six_folder = "C:\\Users\\shpigel-lab\\Desktop\\tiles\\twenty_four"
output_csv = "C:\\Users\\shpigel-lab\\Documents\\DL project\\twenty_four_anomalies.csv"
heatmap_dir = "C:\\Users\\shpigel-lab\\Documents\\DL project\\twenty_four_heatmaps"
os.makedirs(heatmap_dir, exist_ok=True)

with open(BASELINE_PATH, "r") as f:
    baseline = float(f.read().strip())

results = compute_patch_scores(six_folder)
df = pd.DataFrame(results, columns=["Slide", "X", "Y", "RMSE"])
df["Norm_RMSE"] = df["RMSE"] / baseline
df["Abnormal"] = df["Norm_RMSE"] > 1.0

# Save per-slide percentage summary
summary = df.groupby("Slide")["Abnormal"].mean().reset_index()
summary.rename(columns={"Abnormal": "Abnormal_Pct"}, inplace=True)
summary.to_csv(output_csv, index=False)
print(f"✅ CSV saved: {output_csv}")

# === Generate HEATMAPS per slide ===
stride = 256
for slide_id, group_df in df.groupby("Slide"):
    xs = group_df["X"].values
    ys = group_df["Y"].values
    scores = group_df["Norm_RMSE"].values
    grid_w = (max(xs) // stride) + 1
    grid_h = (max(ys) // stride) + 1

    grid = np.zeros((grid_h, grid_w, 3), dtype=np.uint8)
    norm_scores = (scores - np.min(scores)) / (np.max(scores) - np.min(scores) + 1e-8)

    for x, y, s in zip(xs, ys, norm_scores):
        i, j = y // stride, x // stride
        color = (np.array(plt.cm.jet(s)[:3]) * 255).astype(np.uint8)
        grid[i, j] = color

    heatmap = cv2.resize(grid, (grid_w * 10, grid_h * 10), interpolation=cv2.INTER_NEAREST)

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(heatmap)
    ax.set_title(f"{slide_id} Heatmap (Normalized)")
    ax.axis('off')
    sm = plt.cm.ScalarMappable(cmap='jet', norm=plt.Normalize(vmin=np.min(scores), vmax=np.max(scores)))
    sm.set_array([])
    fig.colorbar(sm, ax=ax, fraction=0.046, pad=0.04)
    fig.savefig(os.path.join(heatmap_dir, f"{slide_id}_heatmap.png"), bbox_inches='tight')
    plt.close()

print(f"✅ Heatmaps saved to: {heatmap_dir}")
